# Get top Video Games through Gamespot API


### ABSTRACT


We are working on a video game ratings dataset and will be extracting video game data from 3 different data sources and then will be munging them together to form a consistent dataset. 

## Data

Data gathered from Gamespot API mentioned below. Data consists of 








## Importing Libraries

In [231]:
import requests
import pandas as pd
import json
import os
import rawgpy
import http.client

### DATA SOURCE 1 : USING API

### Importing the dataset using API

In [ ]:
api_key = 'fde9b37127a4f75830670de572f0c948'
url = 'https://chicken-coop.fr/rest/games/[{}]?platform=[{}]&selectors[]=title&selectors[]=genre&selectors[]=score&selectors[]=alsoAvailableOn&selectors[]=image&selectors[]=description'
limit = 5
counter = 0

for offset in range(0, 5000):
    name_only_fields = f'fields name,popularity,cover,similar_games,slug,url;limit {limit};offset {offset};'
    response = requests.post('https://api-v3.igdb.com/games', name_only_fields,
                             headers={"user-key": api_key})
    counter += len(response.json())
print (response)

In [233]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [234]:
# Fetching the tags from the website
url = 'http://www.vgchartz.com/gamedb/games.php?name=&keyword=&console=&region=All&developer=&publisher=&goty_year=&genre=&boxart=Both&banner=Both&ownership=Both&showmultiplat=Yes&results=200&order=Sales&showtotalsales=0&showpublisher=0&showpublisher=1&showvgchartzscore=0&shownasales=0&showdeveloper=0&showcriticscore=0&showpalsales=0&showreleasedate=0&showuserscore=0&showjapansales=0&showlastupdate=0&showothersales=0&showshipped=0'
html = requests.get(url, headers = {'User-Agent':'Mozilla/5.0'})
print(html.status_code)


200


In [243]:
soup = BeautifulSoup(html.content, 'html.parser')
k=[]
rank=[]
gname=[]
publisher=[]
platform=[]
release_date=[]
for tag in soup.find_all('a'):
    if tag['href'].startswith('http://www.vgchartz.com/game/'):
        k.append(tag.get_text().strip())
        data=tag.parent.parent.find_all('td')
        if data!=[]:
            rank.append(np.int32(data[0].string))
            platform.append(data[3].find('img').attrs['alt'].strip(' '))
            publisher.append(data[4].string.strip(' '))
gname = k[10:]
columns = {
    'Rank': rank,
    'Name':gname,
    'Platform':platform,
    'Publisher':publisher
}
df = pd.DataFrame(columns)
print(df.columns)
df = df[[
    'Rank', 'Name', 'Platform',
    'Publisher']]
df.to_csv("vgsales.csv", sep=",", encoding='utf-8', index=False)
print(df)

Index(['Rank', 'Name', 'Platform', 'Publisher'], dtype='object')
     Rank                                      Name Platform  \
0       1                                Wii Sports      Wii   
1       2                         Super Mario Bros.      NES   
2       3                            Mario Kart Wii      Wii   
3       4             PLAYERUNKNOWN'S BATTLEGROUNDS       PC   
4       5                                 Minecraft       PC   
..    ...                                       ...      ...   
195   196                     Red Dead Redemption 2     XOne   
196   197                                   Destiny      PS4   
197   198                                  Tekken 2       PS   
198   199    Uncharted: The Nathan Drake Collection      PS4   
199   200  Super Mario World: Super Mario Advance 2      GBA   

                       Publisher  
0                       Nintendo  
1                       Nintendo  
2                       Nintendo  
3               PUBG Corpo

In [248]:
import json
api_key='f190e9fbed2c2dfe332f94304b0e644b1db12802'
g_reviews=[]
rawTopMoviesJsonData = []
topMovies = []
for i in gname:
    url = 'http://www.gamespot.com/api/games/?api_key='+api_key+'&format=json&filter=name:'+i.replace(" ","-").lower()+',limit:10'
    gameData = requests.get(url)
    print (url)
    rawTopMoviesJsonData.append(gameData.json())
    topMovies += gameData.json()['results']
topMovies=[]
gameData = requests.get(url)
print(gameData.status_code)


http://www.gamespot.com/api/games/?api_key=f190e9fbed2c2dfe332f94304b0e644b1db12802&format=json&filter=name:wii-sports,limit:10


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [258]:
import rawgpy
import json
rawg_ratings = []
genre = []
rawg = rawgpy.RAWG("User-Agent, this should identify your app")
for name in gname:
    results = rawg.search(name)  # defaults to returning the top 5 results
    game = results[0]
    game.populate()
    rawg_ratings.append(game.rating)
print (len(rawg_ratings))

200


In [259]:
print (rawg_ratings)
print ()

[4.2, 4.27, 4.16, 3.4, 4.33, 4.2, 4.31, 4.08, 3.95, 4.33, 4.14, 3.32, 3.36, 3.64, 4.21, 4.39, 3.65, 3.62, 4.34, 4.48, 4.48, 3.62, 4.36, 4.03, 3.92, 4.17, 4.44, 4.32, 3.72, 3.99, 4.52, 4.21, 3.81, 4.41, 4.48, 4.38, 4.16, 4.4, 3.22, 3.66, 2.11, 4.0, 0.0, 3.64, 4.0, 0.0, 4.53, 4.32, 4.17, 4.53, 3.9, 3.9, 4.25, 3.47, 3.64, 4.3, 3.72, 4.43, 4.18, 4.6, 4.33, 4.3, 4.39, 4.0, 4.48, 4.2, 4.34, 4.09, 4.38, 4.07, 4.37, 3.44, 4.69, 4.24, 4.28, 3.88, 4.25, 4.22, 4.32, 4.33, 3.96, 3.28, 4.41, 4.25, 4.22, 4.33, 3.03, 3.9, 3.03, 4.42, 3.92, 4.27, 4.5, 3.81, 4.37, 4.3, 4.24, 4.37, 4.34, 4.41, 3.46, 4.0, 3.93, 4.38, 4.33, 4.24, 3.48, 3.34, 4.47, 4.4, 3.28, 4.1, 4.48, 4.25, 4.38, 3.74, 4.3, 3.56, 3.72, 4.3, 4.35, 3.26, 4.57, 4.33, 3.51, 3.52, 4.41, 3.88, 3.96, 4.21, 4.37, 4.38, 4.35, 3.96, 4.28, 3.39, 3.93, 3.9, 4.24, 4.32, 3.22, 3.98, 0.0, 4.32, 3.79, 3.98, 4.32, 3.93, 4.27, 0.0, 4.34, 3.9, 4.36, 4.0, 4.16, 0.0, 4.36, 4.34, 0.0, 3.2, 4.38, 4.37, 4.18, 4.12, 3.66, 3.95, 4.38, 4.4, 4.08, 4.33, 4.13, 4.33,